In [1]:
from collections import defaultdict
import numpy as np
from pychunkedgraph.graph import ChunkedGraph
from pychunkedgraph.graph.utils.inspect import dfs_print_node
from pychunkedgraph.graph import cache

from pychunkedgraph.graph.edits import add_edges, remove_edges
from pychunkedgraph.graph.utils.context_managers import TimeIt
from pychunkedgraph.graph.operation import MulticutOperation
cg = ChunkedGraph(graph_id="akhilesh-minnie65-1")

/usr/local/lib/python3.7/site-packages/python_jsonschema_objects/__init__.py:53: UserWarning: Schema version http://json-schema.org/draft-04/schema not recognized. Some keywords and features may not be supported.
  self.schema["$schema"]


In [2]:
data = {
        "sources": [["87979484480978348", 717418.8125, 905186.875, 602200]],
        "sinks": [["87979553200440097", 717687.6875, 905449.25, 602200]],
    }
data_dict = {}
for k in ["sources", "sinks"]:
    data_dict[k] = defaultdict(list)
    for node in data[k]:
        node_id = node[0]
        x, y, z = node[1:]
        coordinate = np.array([x, y, z]) / cg.meta._ws_cv.resolution
        atomic_id = cg.get_atomic_id_from_coord(
            coordinate[0],
            coordinate[1],
            coordinate[2],
            parent_id=np.uint64(node_id),
        )

        if atomic_id is None:
            raise ValueError("aha")
        data_dict[k]["id"].append(atomic_id)
        data_dict[k]["coord"].append(coordinate)
data_dict        

{'sources': defaultdict(list,
             {'id': [87979484480978348],
              'coord': [Vec(89677.3515625,113148.359375,15055.0, dtype=float64)]}),
 'sinks': defaultdict(list,
             {'id': [87979553200440097],
              'coord': [Vec(89710.9609375,113181.15625,15055.0, dtype=float64)]})}

In [3]:
# root = cg.get_root(data_dict["sources"]["id"][0])
# dfs_print_node(cg, root, limit=2, sep="    ", count_only=True)
cg.get_cross_chunk_edges_layer(
    [
        [data_dict["sources"]["id"][0], data_dict["sinks"]["id"][0]],
    ]
)

array([2])

In [4]:
cache.clear()
cg.cache = cache.CacheService(cg)

op = MulticutOperation(
    cg,
    user_id="test",
    source_ids=data_dict["sources"]["id"],
    sink_ids=data_dict["sinks"]["id"],
    source_coords=data_dict["sources"]["coord"],
    sink_coords=data_dict["sinks"]["coord"],
    bbox_offset=(240,240,24)
)

with TimeIt("remove edges"):
    done, new_l2_ids = op._apply(operation_id="",timestamp=None)

Start remove edges
   Start get_subgraph
   5.136185169219971
Start run_multicut
   split egdes count 487
   0.06059074401855469
Start get_parents edges
   0.00012993812561035156
l2ids [160037078518727370 160037147238204095]
[160037078518727370 160037147238204095 232113226815373677
 304066642126579478 376195704420366309 448389637900394509
 520728706915481362 591097451094912868 864691139080289280]
****************************************************************************************************
layer 2 [160037078518728137, 160037078518728128, 160037147238205548, 160037147238205536]

----------------------------------------------------------------------------------------------------
new_id 160037078518728137 2 [160037147238205536]

cross {160037078518727370: {2: array([[160037078518727370, 160037147238205536],
       [160037078518727370, 160037147238205548],
       [160037078518727370, 160107447262904909],
       [160037078518727370, 160107447262904971]], dtype=uint64)}}
parent 2321132


----------------------------------------------------------------------------------------------------
new_id 591097451095398527 12 []

----------------------------------------------------------------------------------------------------
new_id 591097451095398528 12 []
****************************************************************************************************
layer 9 []
****************************************************************************************************
layer 10 []
****************************************************************************************************
layer 11 []
31.08988666534424


In [5]:
cg.get_parent(232113226815373677)

304066642126579478

In [6]:
cg.get_children(232113226815373677)

array([160037078518727370, 160107447262904909, 160037147238204095,
       160107447262904971], dtype=uint64)

In [7]:
cg.get_all_parents_dict(160037078518727370)

{3: 232113226815373677,
 4: 304066642126579478,
 5: 376195704420366309,
 6: 448389637900394509,
 7: 520728706915481362,
 8: 591097451094912868,
 12: 864691139080289280}

In [8]:
for i in range(2, 13):
    print(i, new_l2_ids[1][i])

2 [160037078518728137, 160037078518728128, 160037147238205548, 160037147238205536]
3 [232113226815377221, 232113226815377222]
4 [304066642126584246, 304066642126584247]
5 [376195704420398036, 376195704420398037]
6 [448389637900456572, 448389637900456573]
7 [520728706915543600, 520728706915543601]
8 [591097451095398527, 591097451095398528]
9 []
10 []
11 []
12 [864691128455135517, 864691128455135518]


In [9]:
for i in range(2, 5):
    print(i, "*" * 50)
    nodes = np.array(new_l2_ids[1][i], dtype=np.uint64)
    print(nodes)
    print()
    for k,v in cg.get_cross_chunk_edges(nodes).items():
        print("-" * 50)
        print(k, v[i][:,1])
        for k,v in cg.get_cross_chunk_edges(v[i][:,1]).items():
            print()
            print("  ", k, v[i][:,1])
    print()

2 **************************************************
[160037078518728137 160037078518728128 160037147238205548
 160037147238205536]

--------------------------------------------------
160037078518728137 [160037147238205536]

   160037147238205536 [160037078518728137]
--------------------------------------------------
160037078518728128 [160037147238205548 160107447262904909 160107447262904971]

   160037147238205548 [160037078518728128]

   160107447262904909 [160037078518728128]

   160107447262904971 [160037078518728128]
--------------------------------------------------
160037147238205548 [160037078518728128]

   160037078518728128 [160037147238205548 160107447262904909 160107447262904971]
--------------------------------------------------
160037147238205536 [160037078518728137]

   160037078518728137 [160037147238205536]

3 **************************************************
[232113226815377221 232113226815377222]

--------------------------------------------------
23211322681537722

In [10]:
# 2 [160037078518728077, 160037078518728064, 160037147238205488, 160037147238205504]
nodes = np.array([304066642126584207], dtype=np.uint64)
cg.get_parents(nodes)

ValueError: NumPy boolean array indexing assignment cannot assign 0 input values to the 1 output values where the mask is true

In [ ]:
cg.get_children(376195704420398001)

In [ ]:
new_l2_ids[0]

In [ ]:
np.unique(done)

In [ ]:
dfs_print_node(cg, new_l2_ids[1][12][0], limit=2, sep="    ", count_only=True)
dfs_print_node(cg, new_l2_ids[1][12][1], limit=2, sep="    ", count_only=True)